In [1]:
!pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 4.7 MB/s eta 0:00:004.2 MB/s eta 0:00:01
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [3]:
from pyspark import SparkContext

# Stop the existing SparkContext if any
SparkContext.stop(sc)

# Initialize a new SparkContext
sc = SparkContext(master="local", appName="New App")


In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Customer Reviews Analysis") \
    .getOrCreate()

In [5]:
test_data = spark.read.csv("hdfs://localhost:9000/user/hduser/CustomerReviews/test/test.csv", header=True, inferSchema=True)
train_data = spark.read.csv("hdfs://localhost:9000/user/hduser/CustomerReviews/train/train.csv", header=True, inferSchema=True)


In [6]:
# Show first few rows
test_data.show()
train_data.show()

# Show schema
test_data.printSchema()
train_data.printSchema()

+---+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|  1|     mens ultrasheer|This model may be ok for sedentary types, but I'm active and get around alot in my job - consistently found these stockings rolled up down by my ankles! Not Good!! Solution: go with the standard compression stocking, 20-30, stock #114622. Excellent support, stays up and gives me what I need. Both pair of these also tore as I struggled to pull them up all the time. Good riddance/bad investment!|
+---+--------------------+------------------------------------------------------------------------------------------------------------------------------

+---+--------------------+--------------------------------------------------------------+----------------------------------+
|  3|  more like funchuck|"Gave this to my dad for a gag gift after directing ""Nunsense|"" he got a reall kick out of it!"|
+---+--------------------+--------------------------------------------------------------+----------------------------------+
|  5|           Inspiring|                                          I hope a lot of p...|                              null|
|  5|The best soundtra...|                                          I'm reading a lot...|                              null|
|  4|    Chrono Cross OST|                                          "The music of Yas...|               The Girl who Sto...|
|  5| Too good to be true|                                          Probably the grea...|                              null|
|  5|There's a reason ...|                                          There's a reason ...|                              null|


In [7]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

tokenizer = Tokenizer(inputCol="review_text", outputCol="tokens")
hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")

In [14]:
test_data = spark.read.csv(
    "hdfs://localhost:9000/user/hduser/CustomerReviews/test/test.csv", 
    header=True, 
    inferSchema=True, 
    sep=",", 
    quote='"'
)


In [9]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

# Initialize classifier
lr = LogisticRegression(labelCol="label", featuresCol="features")

# Create pipeline
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, lr])


In [19]:
from pyspark.sql import Row

text_data = sc.textFile("hdfs://localhost:9000/user/hduser/CustomerReviews/train/train.csv")
parsed_data = text_data.map(lambda line: Row(
    column1=line.split(',')[0],
    column2=line.split(',')[1],
    # so on...
))

# Convert RDD to DataFrame
train_data = spark.createDataFrame(parsed_data)


2023-09-19 21:58:48,626 WARN impl.BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:658)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:586)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:3025)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:826)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:751)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:381)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:755)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:685)
	

In [21]:
import pandas as pd

pdf = pd.read_csv("file:///home/hduser/Desktop/test.csv")
train_data = spark.createDataFrame(pdf)


AttributeError: 'DataFrame' object has no attribute 'iteritems'